In [52]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [54]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [55]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [56]:
label_map = {0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9}
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
cifar10 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0,1,2,3,4,5,6,7,8,9]]
cifar10_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0,1,2,3,4,5,6,7,8,9]]

In [63]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [64]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")
import datetime  # <1>

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

Training on device cuda.


In [65]:
import time
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model = Net().to(device=device)  # <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()
time1 = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
time2 = time.time()
time3 = time2-time1
print(time3)

2021-04-22 10:44:59.129053 Epoch 1, Training loss 2.0323531330394013
2021-04-22 10:45:15.669233 Epoch 10, Training loss 1.2034821423423259
2021-04-22 10:45:34.179422 Epoch 20, Training loss 1.0340060676302751
2021-04-22 10:45:52.051089 Epoch 30, Training loss 0.9422119404653759
2021-04-22 10:46:09.803114 Epoch 40, Training loss 0.8898053870481604
2021-04-22 10:46:26.460008 Epoch 50, Training loss 0.8442033366931369
2021-04-22 10:46:41.662469 Epoch 60, Training loss 0.8061694959392938
2021-04-22 10:46:57.702884 Epoch 70, Training loss 0.7721621378913255
2021-04-22 10:47:14.467652 Epoch 80, Training loss 0.7460503187935675
2021-04-22 10:47:30.691503 Epoch 90, Training loss 0.7216090995180028
2021-04-22 10:47:46.874608 Epoch 100, Training loss 0.701349069928879
2021-04-22 10:48:03.295068 Epoch 110, Training loss 0.684777773592783
2021-04-22 10:48:20.537671 Epoch 120, Training loss 0.668162872724216
2021-04-22 10:48:37.910204 Epoch 130, Training loss 0.6535478259062828
2021-04-22 10:48:55.

In [66]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

Accuracy train: 0.77
Accuracy val: 0.63


In [69]:
class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [70]:

model = NetDepth(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()
time1 = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
all_acc_dict["depth"] = validate(model, train_loader, val_loader)
time2 = time.time()
time3 = time2-time1
print(time3)

2021-04-22 11:02:30.493220 Epoch 1, Training loss 2.139684502426011
2021-04-22 11:02:49.264088 Epoch 10, Training loss 1.1827155721309546
2021-04-22 11:03:11.092876 Epoch 20, Training loss 0.9419638391041085
2021-04-22 11:03:34.721736 Epoch 30, Training loss 0.8284895798891706
2021-04-22 11:03:57.895880 Epoch 40, Training loss 0.7629766038159276
2021-04-22 11:04:20.864160 Epoch 50, Training loss 0.7176013890358494
2021-04-22 11:04:43.940289 Epoch 60, Training loss 0.6831804167126756
2021-04-22 11:05:06.986145 Epoch 70, Training loss 0.6547993553416503
2021-04-22 11:05:29.956238 Epoch 80, Training loss 0.6310977951797379
2021-04-22 11:05:52.906152 Epoch 90, Training loss 0.6103006697752896
2021-04-22 11:06:15.948479 Epoch 100, Training loss 0.5921452348418248
2021-04-22 11:06:38.931190 Epoch 110, Training loss 0.5756092067341061
2021-04-22 11:07:01.883001 Epoch 120, Training loss 0.5606479425259563
2021-04-22 11:07:24.829865 Epoch 130, Training loss 0.546694454139151
2021-04-22 11:07:47

In [71]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

Accuracy train: 0.82
Accuracy val: 0.68


In [ ]:
#Finish